It does not make sence that i clean the dataset and then test each approach because i wanna check to see which approch is better for the imputation

# Cleaning the Data and test the accuracy

In [37]:
import pandas as pd
# save filepath to variable for easier access
pump_file_path = '../data/processed/pump_dataset.csv'
# read the data and store data in DataFrame titled pomp_data
pump_data = pd.read_csv(pump_file_path) 
# print a summary of the data in water pomp data
pump_data.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [38]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.experimental import enable_iterative_imputer  # This enables IterativeImputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

# Transformer: Replace 0 with random longitude
class LongitudeRandomImputer(BaseEstimator, TransformerMixin):
    def __init__(self, column='longitude', min_val=29.2, max_val=40.4):
        self.column = column
        self.min_val = min_val
        self.max_val = max_val

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.column] = X[self.column].replace(0, np.nan)
        X[self.column] = X[self.column].apply(
            lambda x: np.random.uniform(self.min_val, self.max_val) if pd.isna(x) else x
        )
        return X

# Transformer: Group-based mode imputation with global fallback
class GroupModeImputer(BaseEstimator, TransformerMixin):
    def __init__(self, col, group_cols):
        self.col = col
        self.group_cols = group_cols
        self.global_mode = None

    def fit(self, X, y=None):
        self.global_mode = X[self.col].mode(dropna=True)[0]
        return self

    def transform(self, X):
        X = X.copy()
        try:
            X[self.col] = X.groupby(self.group_cols)[self.col].transform(
                lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x
            )
        except Exception as e:
            print(f"Warning during group imputation of {self.col}: {e}")
        X[self.col].fillna(self.global_mode, inplace=True)
        return X

# Transformer: Iterative imputer for numeric columns
class IterativeNumericImputer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        self.imputer = IterativeImputer(estimator=BayesianRidge(), random_state=42, max_iter=10)

    def fit(self, X, y=None):
        self.imputer.fit(X[self.columns])
        return self

    def transform(self, X):
        X = X.copy()
        X[self.columns] = self.imputer.transform(X[self.columns])
        return X


In [39]:
from sklearn.pipeline import Pipeline

# List of all imputers and their logic
imputation_pipeline = Pipeline(steps=[
    ('longitude_random', LongitudeRandomImputer()),

    ('funder_impute', GroupModeImputer('funder', ['region', 'lga', 'ward'])),
    ('installer_impute', GroupModeImputer('installer', ['region', 'funder'])),
    ('subvillage_impute', GroupModeImputer('subvillage', ['lga', 'ward'])),
    ('public_meeting_impute', GroupModeImputer('public_meeting', ['region', 'ward'])),
    ('scheme_name_impute', GroupModeImputer('scheme_name', ['basin', 'region', 'longitude', 'latitude'])),
    ('scheme_management_impute', GroupModeImputer('scheme_management', ['scheme_name', 'source', 'region'])),
    ('permit_impute', GroupModeImputer('permit', ['region', 'source'])),

    ('iterative_numeric', IterativeNumericImputer(['construction_year', 'population', 'latitude', 'longitude', 'gps_height']))
])


In [40]:
# Separate target from predictors
y = pump_data.status_group
X = pump_data.drop(['status_group'], axis=1)

# Divide data into training and validation subsets
from sklearn.model_selection import train_test_split
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)
X_train_full = X_train_full.copy()
X_valid_full = X_valid_full.copy() 



In [41]:
# Fit and transform the training data
X_train_full = imputation_pipeline.fit_transform(X_train_full)

# Only transform the validation set
X_valid_full = imputation_pipeline.transform(X_valid_full)

In [42]:

# Drop columns
drop_columns = ['id','recorded_by', 'num_private', 'payment']
X_train_full.drop(drop_columns, axis=1, inplace=True)
X_valid_full.drop(drop_columns, axis=1, inplace=True)

# Drop columns with missing values (simplest approach)
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
print(cols_with_missing)
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

[]


In [43]:
missing_counts = X_train_full.isnull().sum().sort_values(ascending=False)
missing_counts


amount_tsh               0
date_recorded            0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
permit                   0
scheme_name              0
scheme_management        0
basin                    0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
subvillage               0
public_meeting           0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
waterpoint_type_group    0
dtype: int64

In [44]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['basin', 'extraction_type_class', 'management_group', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']


## Define Quality for each approach

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestClassifier(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)


In [46]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("Accuracy from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

Accuracy from Approach 1 (Drop categorical variables):
0.7209595959595959


In [47]:
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
0.8051346801346801


In [48]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
0.8002525252525252
